# Sample Summaries  

## Tagged Colony Summaries 

In [21]:
library(dplyr)
library(tidyverse)
library(ggplot2)
library(tidyr)
library(purrr)
library(stringr)

In [22]:
colony <- read_csv("/Users/brookesienkiewicz/Documents/sctld/SCTLD_samples/Sample_Data/CBC_ColonyData.csv", show_col_types = FALSE)

New names:
• `` -> `...1`


In [23]:
# change orbi rapid colony to ofav for now 
unique(colony$Species)
colony[colony$Species=='ORBI','Species']<- "OFAV"
unique(colony$Species)

[1] "SSID" "PAST" "PSTR" "MCAV" "OANN" "ORBI" "DLAB" "MMEA" "OFAV" "CNAT"

[1] "SSID" "PAST" "PSTR" "MCAV" "OANN" "OFAV" "DLAB" "MMEA" "CNAT"

In [24]:
# summary of tagged colonies 
tagged_summarytable<-colony %>%
  group_by(Transect, Species) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0)

    # Add row totals
    tagged_summarytable <- tagged_summarytable %>%
      mutate(Total = rowSums(select(., -Transect)))
    
    # Add column totals
    totals_row <- tagged_summarytable %>%
      select(-Transect) %>%
      summarise(across(everything(), sum)) %>%
      mutate(Transect = "Total") %>%
      select(Transect, everything())
    
    # Combine
    tagged_summarytable <- bind_rows(tagged_summarytable, totals_row)
    tagged_summarytable

Transect,MCAV,OANN,OFAV,PAST,PSTR,SSID,DLAB,MMEA,CNAT,Total
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
BB,6,7,6,6,6,6,0,0,0,37
CBC30N,9,2,1,5,7,8,2,2,0,36
CURLEW,5,1,4,0,5,0,2,0,0,17
HANGMAN,5,5,5,5,6,5,0,0,0,31
LAGOON,10,2,3,9,10,10,2,5,0,51
SR30N,7,2,4,6,9,11,2,6,1,48
Total,42,19,23,31,43,40,8,13,1,220


In [25]:
# summary of immune tagged colonies 
immune_summarytable<-colony %>%
  filter(`immune_y/n`=='y') %>%
  group_by(Transect, Species) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0)

    # Add row totals
    immune_summarytable <- immune_summarytable %>%
      mutate(Total = rowSums(select(., -Transect)))
    
    # Add column totals
    totals_row <- immune_summarytable %>%
      select(-Transect) %>%
      summarise(across(everything(), sum)) %>%
      mutate(Transect = "Total") %>%
      select(Transect, everything())
    
    # Combine
    immune_summarytable <- bind_rows(immune_summarytable, totals_row)
    immune_summarytable

Transect,MCAV,OFAV,PAST,PSTR,SSID,Total
<chr>,<int>,<int>,<int>,<int>,<int>,<dbl>
CBC30N,1,1,3,1,1,7
CURLEW,3,1,0,2,0,6
LAGOON,3,3,3,2,2,13
SR30N,3,2,3,0,3,11
Total,10,7,9,5,6,37


### Bleached

In [37]:
# remove colonies that died before bleaching event 
unique(colony$Date_DocumentedMortality)
class(colony$Date_DocumentedMortality)

[1] "Diseased"               "Healthy"                "12/2/22"               
 [4] "9/25/23"                "12/1/24"                "12/16/23"              
 [7] "4/1/24"                 "4/26/24"                "8/1/24"                
[10] "5/21/22"                "9/26/23"                "12/1/22"               
[13] "5/22/22"                "12/4/22"                "1/11/24"               
[16] "6/21/24"                "5/20/22"                "1/1/24"                
[19] "5/24/23"                "12/3/22"                NA                      
[22] "Could not find 9/26/23"

[1] "character"

In [38]:
# viewing colonies that died in/around 9/2023
colony %>% 
    filter(Date_DocumentedMortality=='5/24/23') %>%
    select(contains("Condition"))
colony %>% 
    filter(str_detect(Date_DocumentedMortality, "^9/.*/23$")) %>%
    select(contains("Condition"))

# do i include dead colonies in 9/23? no? 

062019_Condition,052022_Condition,122022_Condition,092023_Condition,112023_Condition,122023_Condition,012024_Condition,022024_Condition,042024_Condition,062024_Condition,082024_Condition,122024_Condition,062025_Condition
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Healthy,Diseased,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,Not_visited


062019_Condition,052022_Condition,122022_Condition,092023_Condition,112023_Condition,122023_Condition,012024_Condition,022024_Condition,042024_Condition,062024_Condition,082024_Condition,122024_Condition,062025_Condition
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Healthy,Healthy,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Healthy,NA
Healthy,Diseased,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Dead,Not_Visited,Not_Visited,Not_visited,NA
Healthy,Diseased,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,NA
NA,NA,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,NA
Healthy,Healthy,Healthy,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,NA
NA,NA,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,Not_visited
NA,NA,Healthy,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,Not_visited
NA,NA,Healthy,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,NA
NA,NA,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,Not_Visited,Not_visited,NA


In [40]:
clb_colony<-colony %>%
    # remove colonies that died in 2022 and 9/23
  filter(
    !str_detect(Date_DocumentedMortality, "/22") &
    !str_detect(Date_DocumentedMortality, "9/.*/23") &
    !str_detect(Date_DocumentedMortality, "5/.*/23")
  )
     
unique(clb_colony$Date_DocumentedMortality)

[1] "Diseased" "Healthy"  "12/1/24"  "12/16/23" "4/1/24"   "4/26/24" 
 [7] "8/1/24"   "1/11/24"  "6/21/24"  "1/1/24"

In [41]:
# Add bleach status column
clb_summary <- clb_colony %>%
  mutate(
    clb_y_n = if_any(everything(), ~ str_detect(., "CLB|CLP")),
    Status = case_when(
    clb_y_n ~ "Bleached",
    is.na(clb_y_n) ~ "Not Bleached", 
    TRUE ~ "Not Bleached"
    ))
      
# Count by Transect, Species, and Status
summary_tbl <- clb_summary %>%
  group_by(Transect, Species, Status) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0) %>%
  mutate(Total = rowSums(across(-c(Transect, Status))))

# Add row totals per Status
status_totals <- summary_tbl %>%
  select(-Transect) %>%
  group_by(Status) %>%
  summarise(across(everything(), sum)) %>%
  mutate(Transect = "Total") %>%
  select(Transect, everything())

# Add grand total row
grand_total <- summary_tbl %>%
  select(-c(Transect, Status)) %>%
  summarise(across(everything(), sum)) %>%
  mutate(Transect = "Total", Status = "Total") %>%
  select(Transect, Status, everything())

# Combine
clb_summarytable <- bind_rows(summary_tbl, status_totals, grand_total) %>%
  arrange(Transect, Status)

clb_summarytable

Transect,Status,MCAV,OANN,OFAV,PAST,PSTR,SSID,DLAB,CNAT,Total
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
BB,Bleached,4,7,6,6,4,0,0,0,27
BB,Not Bleached,2,0,0,0,2,6,0,0,10
CBC30N,Bleached,2,2,1,5,1,2,1,0,14
CBC30N,Not Bleached,0,0,0,0,0,5,1,0,6
CURLEW,Bleached,2,0,2,0,2,0,1,0,7
CURLEW,Not Bleached,1,1,2,0,0,0,1,0,5
HANGMAN,Bleached,3,4,3,1,6,1,0,0,18
HANGMAN,Not Bleached,2,1,2,4,0,4,0,0,13
LAGOON,Bleached,3,2,0,7,4,3,2,0,21


In [42]:
# repeat for immune 
immune <-clb_summary %>%
  filter(`immune_y/n`=='y')
      
# Count by Transect, Species, and Status
summary_tbl <- immune %>%
  group_by(Transect, Species, Status) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0) %>%
  mutate(Total = rowSums(across(-c(Transect, Status))))

# Add row totals per Status
status_totals <- summary_tbl %>%
  select(-Transect) %>%
  group_by(Status) %>%
  summarise(across(everything(), sum)) %>%
  mutate(Transect = "Total") %>%
  select(Transect, everything())

# Add grand total row
grand_total <- summary_tbl %>%
  select(-c(Transect, Status)) %>%
  summarise(across(everything(), sum)) %>%
  mutate(Transect = "Total", Status = "Total") %>%
  select(Transect, Status, everything())

# Combine
clb_immunesummary <- bind_rows(summary_tbl, status_totals, grand_total) %>%
  arrange(Transect, Status)

clb_immunesummary

Transect,Status,MCAV,OFAV,PAST,PSTR,SSID,Total
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<dbl>
CBC30N,Bleached,1,1,3,1,1,7
CURLEW,Bleached,2,1,0,2,0,5
CURLEW,Not Bleached,1,0,0,0,0,1
LAGOON,Bleached,3,0,3,2,2,10
LAGOON,Not Bleached,0,2,0,0,0,2
SR30N,Bleached,3,1,2,0,3,9
SR30N,Not Bleached,0,1,1,0,0,2
Total,Bleached,9,3,8,5,6,31
Total,Not Bleached,1,3,1,0,0,5


In [45]:
# export 
write.csv(immune_summarytable, 'tables/immune_summarytable.csv', row.names=FALSE)
write.csv(tagged_summarytable, 'tables/tagged_summarytable.csv',row.names=FALSE)
write.csv(clb_summarytable, 'tables/clb_taggedsummary.csv',row.names=FALSE)
write.csv(clb_immunesummary, 'tables/clb_immunesummary.csv',row.names=FALSE)

In [ ]:
## Bleaching Sample Summaries 